In [1]:
import numpy as np
import tensorflow as tf
import ncnn
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# load model
classifier = tf.keras.models.load_model("classifier.h5")
classifier.summary()

Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 128)               65664     
                                                                 
 classification_output (Dens  (None, 7)                903       
 e)                                                              
                                                                 
Total params: 329,223
Trainable params: 329,223
Non-trainable params: 0
_________________________________________________________________


In [3]:
# convert model
! python3 -m keras2ncnn -i classifier.h5 -o ./

Reading and parsing keras h5df file...
Start graph optimizing pass...
	Removing unused nodes...
	Removing squeeze reshape after pooling...
	Refreshing graph...
Converting keras graph to ncnn graph...
Start emitting to ncnn files.
	Emitting param...
	Emitting binary...
Done!


In [4]:
# infer with original model
dummy_input = np.random.random(size=(1, 512)).astype(np.float32)
dummy_output = classifier.predict(dummy_input)
dummy_output

1/1 [==============================] - 0s 143ms/step


array([[ 0.05615039,  0.01951502, -0.01127421, -0.06659301, -0.5894873 ,
         0.5829465 , -0.22231063]], dtype=float32)

In [5]:
# infer with ncnn
with ncnn.Net() as net:
    net.load_param("classifier.param")
    net.load_model("classifier.bin")

    with net.create_extractor() as ex:
        ex.input("input_2_blob", ncnn.Mat(dummy_input).clone())

        _, ncnn_output = ex.extract("classification_output_blob")

ncnn_output

<ncnn.Mat w=7 h=1 d=1 c=1 dims=2 cstep=7 elemsize=4 elempack=1
	refcount=1 data=0x0x6718a40 allocator=0x0>
[[0.0562771, 0.0194222, -0.0111619, ..., -0.589062, 0.582729, -0.222009]]

In [6]:
# compare results
cosine_similarity(dummy_output, np.array(ncnn_output))

array([[0.99999994]], dtype=float32)